<a href="https://colab.research.google.com/github/Keltings/Soccer/blob/main/Group7WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#downlaod the html of a page
import requests
#parsing html
from bs4 import BeautifulSoup

In [ ]:
#define the url to start scraping
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
#standings_url = "https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats"


In [ ]:
#downlaod the page
data = requests.get(standings_url)
#data.text

In [ ]:
#initialze beutifulSoup using html
soup = BeautifulSoup(data.text)

In [ ]:
# select the table using the css selector
standings_table = soup.select("overall.stats_table")[0]#select the first element in the list

#Basically removed alot of the extra html
#standings_table

IndexError: list index out of range

In [ ]:
# find all of the a tags inside the table
links = standings_table.find_all("a")

# now get the href property of each link
#list comprehension
links = [l.get('href') for l in links]

#filter the links to get the squad links to give us stats for each individual squad
links = [l for l in links if '/squads/' in l]
links

Note that the `select` method uses css selectors which gives a lot of flexibility to select different elements, classes, ids, etc. Whereas `find_all` only find tags.

In [ ]:
# turn our links into full urls
team_url = [f'https://fbref.com{l}' for l in links]
team_url

In [ ]:
import pandas as pd
#for only Man-city
team_url=team_url[0]
data = requests.get(team_url)
matches = pd.read_html(data.text, match='Scores & Fixtures')
matches[0].shape

### get matches shooting stats with requests and pandas

In [ ]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')

#list comprehension
links = [l.get('href') for l in links]

#filter the links to get the squad links to give us stats for each individual squad
links = [l for l in links if l and 'all_comps/shooting/' in l]
links

In [ ]:
data=requests.get(f"https://fbref.com{links[0]}")
shooting = pd.read_html(data.text, match='Shooting')[0]

### Cleaning and merging scraped data in pandas

In [ ]:
shooting.head()

In [ ]:
# remove multilevel indexing ie columns with two names
shooting.columns = shooting.columns.droplevel()
shooting.shape

In [ ]:
team_data = matches[0].merge(shooting[['Date', 'Sh', 'SoT', 'Dist', 'FK', 'PK', 'PKatt']], on='Date')
team_data.shape

In [ ]:
print(matches[0].shape)
print(shooting.shape)

### Scaling the Method Up

scrape data for multiple teams for multiple for multiple years

In [ ]:
# set up a list of the years we want to scrape
years = list(range(2023,2019, -1))
years

In [ ]:
# initialize a list to contain several dataframes such that
# each df contains the match logs for one team for each season
all_matches = []

# find the url we want to start on
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"


In [ ]:
import time
# loop through all the years
#perform all the steps from our earlier codes
#the codes we used to get  manchester
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)

    #the table contains all of our teams individual links
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get('href') for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    # turn from relative to absolute links
    team_urls = [f"https://fbref.com{l}" for l in links]

    #grab the url for the previous seasons
    previous_season = soup.select('a.prev')[0].get('href')
    standings_url = f'https://fbref.com/{previous_season}'

    #loop through each of the team urls
    for team_url in team_urls:
        #individually scarpe the match logs for each team
        #splitting on / we get the names of the team
        team_name = team_url.split('/')[-1].replace('-Stats', '').replace("-", " ")
        #print(team_url)
        #print(team_url.split('/')[-1])
        #print(team_name)

        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

        #get for the shootings table
        soup = BeautifulSoup(data.text)
        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        shooting = pd.read_html(data.text, match='Shooting')[0]
        shooting.columns=shooting.columns.droplevel()

        try:
          team_data = matches.merge(shooting[['Date', 'Sh', 'SoT', 'Dist', 'FK', 'PK', 'PKatt']], on='Date')

      # filter our data for only competitoion was the premier league
        # sometimes for some teams the shooting stats arent available so we wrap in
        # a try and exception handle
        # to not slow down the website
        except ValueError:
          continue
        team_data = team_data[team_data['Comp'] == 'Premier League']
        team_data['Season'] = year
        team_data['Team'] = team_name
        all_matches.append(team_data)

        # to not scrape too quickly
        time.sleep(3)



In [ ]:
print(all_matches)

In [ ]:
#combine all the dfs
all_matches

# all column names to be lower case
all_matches.columns = [c.lower() for c in all_matches.columns]

# write my data to csv file
all_matches.to_csv('matches.csv')
